# Sw00pGenerator3000

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import peakutils as pu
from plotly.subplots import make_subplots

from jupyter_dash import JupyterDash 
from dash import Dash, dcc, html, Input, Output, callback

themes = ["plotly_white", "plotly_dark", "ggplot2", "simple_white"]
pio.templates.default = themes[0]

In [2]:
token = "pk.eyJ1IjoiYWlieWJydW0iLCJhIjoiY2xhdmRldnhyMDRkNTNybWg0NTl6eHgwZSJ9.phqQCg9UWIXx5wFkc_i0kg"

landing_df = pd.read_csv('../data/landing/J2.csv')
landing_df.head()

,time,lat,lon,elevation,horz_distance_ft,horz_distance_m,vert_speed_mph,horz_speed_mph,vert_speed_km/u,horz_speed_km/u,heading,dive_angle
0,748.0,56.181916,9.044068,741.207349,69440.401297,21165.434315,0.961882,44.490233,1.548,71.600099,19.64331,1.238546
1,748.2,56.181948,9.044090,741.748688,69452.942948,21169.257010,-2.751431,41.116076,-4.428,66.169919,22.18640,-3.828447
2,748.4,56.181977,9.044113,743.218504,69464.493513,21172.777623,-5.726556,37.811868,-9.216,60.852311,24.83665,-8.611923
3,748.6,56.182003,9.044137,745.390420,69475.115828,21176.015304,-7.538474,35.000565,-12.132,56.327957,28.34259,-12.154771
4,748.8,56.182026,9.044161,747.608268,69484.910519,21179.000726,-7.806907,32.366980,-12.564,52.089612,30.92760,-13.560711


In [3]:
def get_dic(metric):
    return {
        'Elevation':        {'col': landing_df.elevation, 
                             'color': '#636EFA', 
                             'metric': "ft",
                             'hovertemplate': 'Elevation: %{y:.2f} ft <extra></extra>'},
        'Horizontal speed': {'col': landing_df['horz_speed_km/u'] if metric == "km/u" else landing_df['horz_speed_mph'], 
                             'color': '#FF0B0B', 
                             'metric': metric,
                             'hovertemplate': 'Horz speed: %{y:.2f} ' + metric + '<extra></extra>'},
        'Vertical speed':   {'col': landing_df['vert_speed_km/u'] if metric == "km/u" else landing_df['vert_speed_mph'],
                             'color': '#00CC96', 
                             'metric': metric,
                             'hovertemplate': 'Vert speed: %{y:.2f} ' + metric + '<extra></extra>'},
        'Dive angle':       {'col': landing_df.dive_angle,
                             'color': '#AB63FA', 
                             'metric': 'deg',
                             'hovertemplate': 'Dive angle: %{y:.2f}° <extra></extra>'},
    }


app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Sw00pGenerator3000", style={'padding': 10}),
    html.Div(children=[
        html.Div(children=[
            dcc.Dropdown(
                ['Elevation', 'Horizontal speed', 'Vertical speed', 'Dive angle'],
                ['Elevation', 'Horizontal speed', 'Vertical speed'],
                multi=True,
                id='y_drop'
            ),
            dcc.Dropdown(['km/u', 'mph'], 'km/u', id='speed_metric', style={'margin-top': 10}),
            dcc.Dropdown(['m', 'ft'], 'm', id='distance_metric', style={'margin-top': 10}),
            dcc.Graph(
                id='first_graph',
            )
        ], style={'flex': 1, 'flex-basis': 10}),        
    ], style={'padding': 10,'display': 'flex', 'flex-direction': 'row'})   
])

@app.callback(
    Output('first_graph', 'figure'),
    Input('y_drop', 'value'),
    Input('speed_metric', 'value'),
    Input('distance_metric', 'value'))
def update_figure(selected, speed_metric, distance_metric):
    dic = get_dic(speed_metric)
    d_metric = landing_df['horz_distance_m'] if distance_metric == "m" else landing_df['horz_distance_ft']
    
    plotly_data = []
    layout_kwargs = {'xaxis': {'domain': [0.06 * (len(selected)-1), 1], 'title': 'Horizontal distance (' + distance_metric + ')'}}

    for i, s in enumerate(selected):
        axis_name = 'yaxis' + str(i + 1) * (i > 0)
        yaxis = 'y' + str(i + 1) * (i > 0)
        plotly_data.append(go.Scatter(go.Scatter(       
            x=d_metric,
            y=dic[s]['col'],
            name=s,
            mode='lines',                                                 
            line = dict(color=dic[s]['color'], width=1.2),  
            showlegend=False,
            hovertemplate = dic[s]['hovertemplate'],
            ),                            
        ))
        layout_kwargs[axis_name] = {
            'position': i * 0.06,  'side': 'left', 'title': s +' (' + dic[s]['metric'] + ')', 'titlefont': {'size': 10, 'color': dic[s]['color']}, 'title_standoff': 0,
            'anchor': 'free', 'tickfont': {'size': 10, 'color': dic[s]['color']}, 'showgrid':False,
        }

        plotly_data[i]['yaxis'] = yaxis
        if i > 0:
            layout_kwargs[axis_name]['overlaying'] = 'y'

    fig = go.Figure(data=plotly_data, layout=go.Layout(**layout_kwargs))
    fig.update_layout(hovermode="x unified")
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, mode="inline")

In [24]:
def plt_peaks_lows():
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"colspan": 2}, None],
               [{}, {}]], subplot_titles=("Elevation","Horizontal speed", "Vertical speed"))
        
    elevation_peaks = pu.indexes(landing_df.elevation, thres=0.1, min_dist=1)
    elevation_lows = pu.indexes(-landing_df.elevation, thres=0.5, min_dist=1)
    
    horz_speed_peaks = pu.indexes(landing_df.horz_speed_mph, thres=0.1, min_dist=1)
    horz_speed_lows = pu.indexes(-landing_df.horz_speed_mph, thres=0.5, min_dist=1)
    
    vert_speed_peaks = pu.indexes(landing_df.vert_speed_mph, thres=0.5, min_dist=1)
    vert_speed_lows = pu.indexes(-landing_df.vert_speed_mph, thres=0.5, min_dist=1)
    
    fig.add_trace(go.Scatter(x=landing_df['horz_distance_m'], y=landing_df['elevation'], showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(mode='markers', x=landing_df['horz_distance_m'][elevation_peaks], y=landing_df['elevation'][elevation_peaks],
        marker=dict(color='Green', size=10), showlegend=False
    ), row=1, col=1)
    fig.add_trace(go.Scatter(mode='markers', x=landing_df['horz_distance_m'][elevation_lows], y=landing_df['elevation'][elevation_lows],
        marker=dict(color='Red', size=10),showlegend=False
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=landing_df['horz_distance_m'], y=landing_df['horz_speed_mph'], showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(mode='markers', x=landing_df['horz_distance_m'][horz_speed_peaks], y=landing_df['horz_speed_mph'][horz_speed_peaks],
        marker=dict(color='Green', size=10), showlegend=False
    ), row=2, col=1)
    fig.add_trace(go.Scatter(mode='markers', x=landing_df['horz_distance_m'][horz_speed_lows], y=landing_df['horz_speed_mph'][horz_speed_lows],
        marker=dict(color='Red', size=10),showlegend=False
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(x=landing_df['horz_distance_m'], y=landing_df['vert_speed_mph'], showlegend=False), row=2, col=2)
    fig.add_trace(go.Scatter(mode='markers', x=landing_df['horz_distance_m'][vert_speed_peaks], y=landing_df['vert_speed_mph'][vert_speed_peaks],
        marker=dict(color='Green', size=10), showlegend=False
    ), row=2, col=2)
    fig.add_trace(go.Scatter(mode='markers', x=landing_df['horz_distance_m'][vert_speed_lows], y=landing_df['vert_speed_mph'][vert_speed_lows],
        marker=dict(color='Red', size=10),showlegend=False
    ), row=2, col=2)
    
    
    fig.update_layout(height=800)
    fig.show()
    
    
plt_peaks_lows()

In [25]:
def get_top_of_turn():
    return pu.indexes(landing_df.elevation, thres=0.01, min_dist=1)[0]

def get_stop():
    offset = 1.5
    horz_speed_peaks = pu.indexes(landing_df.horz_speed_mph, thres=0.1, min_dist=1)
    horz_speed_lows = pu.indexes(-landing_df.horz_speed_mph, thres=0.5, min_dist=1)
    return [l for l in horz_speed_lows if l > horz_speed_peaks[-1] and landing_df.horz_speed_mph[l] < offset][0]

def get_max_horz_speed():
    return landing_df.idxmax().horz_speed_mph

In [31]:
# Impossible!!!

def plt_touch_down():
    horz_speed_peaks = pu.indexes(landing_df.horz_speed_mph, thres=0.6, min_dist=1)
    horz_speed_lows = pu.indexes(-landing_df.horz_speed_mph, thres=0.5, min_dist=1)
    
    new_df = landing_df.iloc[horz_speed_peaks[-1]:get_stop()+1].reset_index(drop=True)
    s_speed_peaks = pu.indexes(np.diff(new_df.horz_speed_mph, 2), thres=0.1, min_dist=1)
    s_speed_lows = pu.indexes(-np.diff(new_df.horz_speed_mph, 2), thres=0.1, min_dist=1)
    
    speed = [new_df['horz_speed_mph'][s_speed_peaks[i+1]] - new_df['horz_speed_mph'][s_speed_peaks[i]] for i in range(0, len(s_speed_peaks)-1)]
    
    sorted_speed = sorted(range(len(speed)), key=lambda k: speed[k])
    print(sorted_speed)
    l = []
    for i in sorted_speed:
        if new_df['vert_speed_mph'][s_speed_peaks[i]] <= -1:
            l.append(i)
    print(l)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=new_df['horz_distance_m'], y=new_df['horz_speed_mph'], showlegend=False))
    fig.add_trace(go.Scatter(x=new_df['horz_distance_m'], y=new_df['vert_speed_mph'], showlegend=False))
    fig.add_trace(go.Scatter(mode='markers', x=new_df['horz_distance_m'][s_speed_peaks], y=new_df['horz_speed_mph'][s_speed_peaks],
        marker=dict(color='Green', size=6),showlegend=False
    ))
    fig.add_trace(go.Scatter(mode='markers', x=new_df['horz_distance_m'][s_speed_peaks], y=new_df['vert_speed_mph'][s_speed_peaks],
        marker=dict(color='Green', size=6),showlegend=False
    ))
    fig.add_trace(go.Scatter(mode='markers', x=[new_df['horz_distance_m'][s_speed_peaks[l[0]]]], y=[new_df['horz_speed_mph'][s_speed_peaks[l[0]]]],
        marker=dict(color='Blue', size=6),showlegend=False
    ))
    fig.show()
    
plt_touch_down()

[7, 4, 9, 6, 8, 3, 1, 2, 10, 5, 0]
[5]


In [41]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.4, 0.6], subplot_titles=("Side view of flight path","Speed during swoop"))

# Side view of flight path
fig.add_trace(go.Scatter(x=landing_df['horz_distance_m'].iloc[get_top_of_turn():get_stop()+1], y=landing_df['elevation'].iloc[get_top_of_turn():get_stop()+1], showlegend=False,), row=1, col=1)
fig.add_trace(go.Scatter(x=[landing_df['horz_distance_m'][get_top_of_turn()]], y=[landing_df['elevation'][get_top_of_turn()]], 
                         showlegend=False, text=["Top of turn"], textposition="top right", mode="markers+text",), row=1, col=1)
fig.add_trace(go.Scatter(x=[landing_df['horz_distance_m'][get_stop()]], y=[landing_df['elevation'][get_stop()]], 
                         showlegend=False, text=["stop"], textposition="top left", mode="markers+text",), row=1, col=1)


# Speed during swoop
fig.add_trace(go.Scatter(x=landing_df['horz_distance_m'].iloc[get_max_horz_speed():get_stop()+1], y=landing_df['horz_speed_mph'].iloc[get_max_horz_speed():get_stop()+1], showlegend=False,), row=1, col=2)
fig.add_trace(go.Scatter(x=[landing_df['horz_distance_m'][get_max_horz_speed()]], y=[landing_df['horz_speed_mph'][get_max_horz_speed()]], 
                         showlegend=False, text=["Max Horz Speed"], textposition="top right", mode="markers+text",), row=1, col=2)
fig.add_trace(go.Scatter(x=[landing_df['horz_distance_m'][get_stop()]], y=[landing_df['horz_speed_mph'][get_stop()]], 
                         showlegend=False, text=["stop"], textposition="top left", mode="markers+text",), row=1, col=2)

fig['layout']['xaxis']['title']='Horizontal distance (m)'
fig['layout']['xaxis2']['title']='Horizontal distance (m)'
fig['layout']['yaxis']['title']='Height (ft)'
fig['layout']['yaxis2']['title']='Horizontal speed (mph)'
fig.update_layout(hovermode="x unified")
fig.show()

In [34]:
fig = px.line_mapbox(landing_df.iloc[get_top_of_turn():get_stop()+1], lat="lat", lon="lon", zoom=16)
fig.update_layout(mapbox_style="satellite-streets", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [6]:
# Trying cesium

external_css = ['https://cesium.com/downloads/cesiumjs/releases/1.76/Build/Cesium/Widgets/widgets.css']
external_scripts = [{'src':'https://cesium.com/downloads/cesiumjs/releases/1.76/Build/Cesium/Cesium.js'}]

app = JupyterDash(__name__, 
                title='Cesium Test',
                external_scripts=external_scripts,
                external_stylesheets=external_css)

app.layout = html.Div(id='blah', children=['Testing...', html.Div(id='cesiumContainer')])

app.clientside_callback(
    '''
    function(id) {
        Cesium.Ion.defaultAccessToken = "any_code_works";
        var viewer = new Cesium.Viewer(id);
        return true;
    }
    ''',
    Output('cesiumContainer', 'data-done'),
    Input('cesiumContainer', 'id')
)

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash app running on http://127.0.0.1:8050/
